# Splitting into train, dev, and test set

Splits are defined according to criteria in `20210527_initial_eda.ipynb`:

- remove users with less $< 5$ ratings
- remove books with less $< 5$ ratings
- training set ranges from August 2006 to June 2017
- dev set ranges from July to August 2017
- test set ranges from September to October 2017

In [1]:
import json
from pathlib import Path

import pandas as pd
import seaborn as sns